In [2]:
# notebook doing extraction of OLINK data

import pickle as pkl
import pandas
import pandas as pd
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_bool_dtype
import numpy as np
import pickle as pkl 
import os,re
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn
import statsmodels
import statsmodels.formula.api as smf
import seaborn as sns
import xarray
#import GPy
from scipy.stats import chi2_contingency
import numbers,numpy.random
from statsmodels.multivariate.pca import PCA
from sklearn.datasets import make_regression
import warnings
warnings.filterwarnings(action='once')
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.set_option('use_inf_as_na',True)
#%load_ext autoreload
#%autoreload 2

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [4]:
os.chdir('/data/biobank/Proteomics/OLINK')
OLINK_data=pd.read_csv('olink_data2.txt',sep='\t')

eids_OLINK=OLINK_data[OLINK_data['ins_index']==2]['eid'].unique()
eids_SIMOA=data.index.unique()
common_eids = np.union1d(eids_OLINK,data.index)
np.savetxt('subjects_OLINK.txt',common_eids,fmt='%10.0f')

OLINK=pd.DataFrame(index=np.unique(OLINK_data['eid']))
Plists={}
Plists[0]=[]
Plists[2]=[]
Plists[3]=[]

Plists['all'] = OLINK_data['protein_id'].unique()

for a in Plists['all']:
    for b in [0,2,3]:
        Plists[b].append('P_' + str(a) + '-' + str(b))
        els=(OLINK_data['protein_id']==a)&(OLINK_data['ins_index']==b)
        outsuff='0'
        if b==2:
            outsuff='pre'
        elif b==3:
            outsuff='post'
        OLINK.loc[OLINK_data.loc[els,'eid'],'P_' + str(a) + '_' + outsuff] = OLINK_data.loc[els,'result'].values
        
OLINK_meta=pd.read_csv('ukb672393.csv')
OLINK_meta=OLINK_meta.set_index('eid')
OLINK_meta=OLINK_meta.loc[data.index,:]

NameError: name 'data' is not defined

In [16]:
OLINK_data[(OLINK_data['eid']==1024229) & (OLINK_data['ins_index']==1)]

,eid,ins_index,protein_id,result


In [33]:
len(OLINK_data['ins_index'].unique())

3

In [ ]:
OLINK.columns=[a.replace('_3','_post') for a in OLINK.columns]
OLINK.columns=[a.replace('_2','_pre') for a in OLINK.columns]

In [ ]:
# eids_OLINK=OLINK_data[OLINK_data['ins_index']==2]['eid'].unique()
# eids_SIMOA=data.index.unique()
# common_eids = np.union1d(eids_OLINK,data.index)

In [ ]:
# Plists['all'] = list(OLINK_data['protein_id'].unique())
# #  participants with too many missing vals
# parts_t0=(OLINK.loc[:,Plists[0]].notnull().sum(axis=1)>500)
# parts_t23=(OLINK.loc[:,Plists[2]].notnull().sum(axis=1)>500) & (OLINK.loc[:,Plists[3]].notnull().sum(axis=1)>500)

# common_eids=np.array([ a in data.index for a in OLINK.index])
# remain_eids = (~common_eids & parts_t0)
# common_eids = (common_eids & parts_t23 & parts_t0)
# common_eids_0 = (common_eids & parts_t23 )

In [ ]:
coding = pd.read_csv('coding143.tsv',sep='\t',index_col=0)
coding.loc[:,'Protein']=np.nan
coding.loc[:,'Protein']=coding['meaning'].str.extract('(.*);').values

OLINK.columns=[a.replace('-post','_post') for a in OLINK.columns]
OLINK.columns=[a.replace('-pre','_pre') for a in OLINK.columns]

repl={}
for a in coding.index:
    cols=OLINK.columns[OLINK.columns.str.contains('_'+str(a)+'-')]
    
    for col in cols:
        repl[col]=col.replace('_'+str(a)+'-','_'+coding.loc[a,'Protein']+'_')
OLINK=OLINK.rename(columns=repl)
OLINK.columns=[a.replace('_2','_pre') for a in OLINK.columns]
OLINK.columns=[a.replace('_3','_post') for a in OLINK.columns]
OLINK.to_csv('OLINK_data.csv')


In [ ]:

cols=(OLINK.loc[:,OLINK.columns.str.contains('_pre')].notna().sum()>20)
cols=list(cols[cols].index.values)

eids_OLINK=(OLINK[cols].notna().sum(axis=1))>0
eids_OLINK=eids_OLINK[eids_OLINK].index.values

eids_SIMOA=data.index.unique()

common_eids = np.union1d(eids_OLINK,data.index)

cols = cols + [a.replace('_pre','_post') for a in cols] + [a.replace('_pre','_0') for a in cols]

OLINK.loc[subjs_OLINK,cols].to_csv('/data/biobank/Proteomics/OLINK/OLINK_data_COVID.csv')